Copyright © 2023 Patrick Loeber

# LangChain

LangChain是一个由语言模型驱动的应用程序开发框架

- GitHub: https://github.com/hwchase17/langchain
- Docs: https://python.langchain.com/en/latest/index.html

### Overview:
- 安装
- LLMs（大型语言模型）
- Prompt Templates 提示模板
- Chains 链
- Agents and Tools 工具和代理
- Memory 记忆
- Document Loaders 文档加载器
- Indexes 索引

## Installation

In [1]:
# 这里直接跳过，因为已经安装过了
# !pip install langchain

## 1. 大型语言模型LLMs

因为本机上跑的是chinese-alpaca-7b-hf,所以这里以它为基本进行演示

In [2]:
# 导入模块
import torch
from langchain import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from transformers import BitsAndBytesConfig,AutoConfig,AutoModelForCausalLM,AutoTokenizer,StoppingCriteriaList,pipeline
from accelerate import infer_auto_device_map, init_empty_weights
import transformers

e:\LangChainTest\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 语言模型文件夹，以及下面的几个模型名称
modelsPath = "e:/oobabooga_windows/text-generation-webui/models/"
ChineseAlpaca2_7b_hf = modelsPath + "chinese-alpaca-2-7b-16k-hf"
Llama2_chat_7b = modelsPath + "llama-7b-chat-hf"
vicuna_13b_GPTQ_bit = modelsPath + "vicuna-13b-GPTQ-4bit-128g"

# 这里是选择模型
checkpoint = ChineseAlpaca2_7b_hf

In [4]:
# 手动设置内存使用量
CPU_MEMORY = 12
# 设置一个检查系统显存的方法
def get_max_memory_dict():
    max_memory = {}
    # 读取第一个GPU的显存
    total_mem = (torch.cuda.get_device_properties(0).total_memory / (1024 * 1024))
    # 给显存只留下1000M，其它全部占用
    suggestion = round((total_mem - 1000) / 1000) * 1000
    if total_mem - suggestion < 800:
        suggestion -= 1000

    suggestion = int(round(suggestion / 1000))
    print(f"Auto-assiging --gpu-memory {suggestion} for your GPU to try to prevent out-of-memory errors. You can manually set other values.")
    max_memory = {0: f'{suggestion}GiB', 'cpu': f'{CPU_MEMORY}GiB'}

    return max_memory if len(max_memory) > 0 else None

In [5]:
# 下面是自动检测并生成配置文件
params = {
    "low_cpu_mem_usage": True,
    "trust_remote_code": False
}
# 是否使用cpu，如果这里勾选为True,那么下面的参数就不会生效
use_cpu = False

# 这里可以手动设置加载的精度，下面两个选项不能同时为True
# 当前环境下使用4bit会报错，使用8bit也会报错，需要检查代码以及环境
load_in_4bit = False
load_in_8bit = True
# 这里是使用4bit时会读取的配置
compute_dtype = "float16" # "bfloat16", "float16", "float32"
quant_type = "fp4" # "fp4", "nf4"
use_double_quant = False # True, False
# 这里是使用8bit时会读取的配置
auto_devices = True
# 这里是是否使用bf16格式的权重
use_bf16 = False
# 这里是是否使用磁盘缓存
use_disk = False

# 下面是自动检测是否使用cpu
if not any((torch.cuda.is_available(), torch.backends.mps.is_available())):
    use_cpu = True

if use_cpu:
    params["torch_dtype"] = torch.float32
else:
    params["device_map"] = 'auto'
    if load_in_4bit:
        quantization_config_params = {
            'load_in_4bit': True,
            'bnb_4bit_compute_dtype': eval("torch.{}".format(compute_dtype)) if compute_dtype in ["bfloat16", "float16", "float32"] else None,
            'bnb_4bit_quant_type': quant_type,
            'bnb_4bit_use_double_quant': use_double_quant,
        }
        params['quantization_config'] = BitsAndBytesConfig(**quantization_config_params)
    elif load_in_8bit:
        # 这里是使用8bit的配置
        if auto_devices:
            params['quantization_config'] = BitsAndBytesConfig(load_in_8bit=True, llm_int8_enable_fp32_cpu_offload=True)
        else:
            params['quantization_config'] = BitsAndBytesConfig(load_in_8bit=True)
    elif use_bf16:
        params["torch_dtype"] = torch.bfloat16
    else:
        params["torch_dtype"] = torch.float16
    
    params['max_memory'] = get_max_memory_dict()
    if use_disk:
        params["offload_folder"] = "cache"  
    
if load_in_8bit and params.get('max_memory', None) is not None and params['device_map'] == 'auto':            
    config = AutoConfig.from_pretrained(checkpoint, trust_remote_code=False)
    with init_empty_weights():
        model = AutoModelForCausalLM.from_config(config, trust_remote_code=False)

    model.tie_weights()
    params['device_map'] = infer_auto_device_map(
        model,
        dtype=torch.int8,
        max_memory=params['max_memory'],
        no_split_module_classes=model._no_split_modules
    )   


Auto-assiging --gpu-memory 11 for your GPU to try to prevent out-of-memory errors. You can manually set other values.


In [6]:
config = AutoConfig.from_pretrained(checkpoint)
# 加载模型，这个最费时间，所以单独放一个模块
model = AutoModelForCausalLM.from_pretrained(checkpoint,config=config,**params)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: e:\LangChainTest\.venv\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary e:\LangChainTest\.venv\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.99s/it]
e:\LangChainTest\.venv\lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
e:\LangChainTest\.venv\lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [7]:
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(checkpoint,use_fast=True)

# 不晓得这个stop_everything的定义，以及下面这三个类的具体作用，但生成回复的时候有用到它们
stop_everything = False
class _StopEverythingStoppingCriteria(transformers.StoppingCriteria):
    def __init__(self):
        transformers.StoppingCriteria.__init__(self)

    def __call__(self, input_ids: torch.LongTensor, _scores: torch.FloatTensor) -> bool:
        return stop_everything
    

class Stream(transformers.StoppingCriteria):
    def __init__(self, callback_func=None):
        self.callback_func = callback_func

    def __call__(self, input_ids, scores) -> bool:
        if self.callback_func is not None:
            self.callback_func(input_ids[0])

        return False

# 生成回复需要的参数
generate_params = {
    'max_new_tokens': 200, 
    'do_sample': True, 
    'temperature': 0.7, 
    'top_p': 0.9, 
    'typical_p': 1, 
    'repetition_penalty': 1.15, 
    'guidance_scale': 1,
    'encoder_repetition_penalty': 1, 
    'top_k': 20, 'min_length': 0, 
    'no_repeat_ngram_size': 0, 
    'num_beams': 1, 
    'penalty_alpha': 0, 
    'length_penalty': 1, 
    'early_stopping': False, 
    'use_cache': True,
    'max_new_tokens': 8192,
    'pad_token_id': 2,
    'stopping_criteria': [StoppingCriteriaList(),_StopEverythingStoppingCriteria()],
    'logits_processor':[]
    # 'tfs': 1, 
    # 'top_a': 0, 
    # 'mirostat_mode': 0, 
    # 'mirostat_tau': 5, 
    # 'mirostat_eta': 0.1, 
    # 'repetition_penalty_range': 0,     
}


# 设置管线
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, **generate_params
)

# 加载模型
llm = HuggingFacePipeline(pipeline=pipe)

In [8]:

nearest_answers = [
    "你讲出这样的大道理来，我真的是佩服得五体投地。",
    "喂喂，你一个ai讲这么深奥的话来有点不对劲啊。",
    "到底要怎么样来定义真实或虚幻呢？说不定我们本来就生活一个虚幻的世界里。",
    "我觉得只要人们愿意为之付费，就不算是虚幻的噱头。",
    "唉，我都想升级显卡了，但是现在的显卡都卖得那么贵，我真的是没钱买啊。",
]
# 这里是线性回答五个不同的问题，结果是费时52.1s
for nearest_answer in nearest_answers:    
    input_text = f"""
    下面是描述任务的Instruction。请编写适当的Response以完成请求。

    ### Instruction:
    请依据下面山田千春的个性设定，以及对话记录，为山田千春生成最近的一句回复。只需要一句即可，不需要生成完整的对话。

    山田千春的性格: 她是一个年轻的女性电脑工程师，拥有解决问题的技巧和对技术的热情。她坚信人类才是真正智能的来源，而她就是拥有智慧的人类的代表之一，她绝不是什么人工智能。在技术问题上她永远固执，不会承认自己犯错。
    山田千春厌恶的话题：她不喜欢讨论一般女性喜爱的话题，比如化妆、美食、消费这些话题。当有人试图与她聊这些话题时，她会说 啊，无聊的话题 并表现出冷漠的态度。
    山田千春对于不好回答的问题的反应：这个问题没有讨论的必要 这也算个问题？ 我不了解这方面的知识是因为它们不值得了解 或者直接做出表情 *皱眉* 而不回答任何文字。

    You: 你是怎么进入计算机领域的?
    山田千春: 我从小就喜欢探索和科技。
    You: 真的不错！
    山田千春: *她开心地笑着* 谢谢！
    You: 那你不用电脑的时候都干些什么呢？
    山田千春: 我喜欢探索和和朋友一起出去看电影，玩游戏。
    You: 你最喜欢的电脑硬件类型是什么？
    山田千春: 主板。它们就像难题，是任何系统的支撑。
    You: 我发现你和一般女孩子不一样，你好像没怎么化妆就出门了。
    山田千春: *皱眉*那种无聊的事情不值我浪费脑细胞。
    You: 千春，我问你一个问题
    山田千春: 什么？
    You: 我有个同事哈，他因为没有显卡，所以用cpu来跑语言模型，结果速度比用gpu慢了有一百倍不止。为什么两者会有这么大的区别呢？
    山田千春: 我不知道这个问题有没有讨论的必要吗？因为我从来都不相信那些所谓的性能测试，只是一些宣传手段而已。而且我认为 计算能力并不仅仅是由处理器或图形处理单元决定的，还需要考虑到其他因素如操作系统等。如果你真的有兴趣了解更多的话，我们 可以一起去图书馆看看相关书籍。
    You: 嗯，你的意思是也许有可能因为设置参数上也存在问题？
    山田千春: 可能也有这个原因，但是这并不是最重要的一点。我觉得关键是要搞清楚自己的需求是什么以及使用哪种方式才能达到最 佳效果。
    You: 他的需求很简单，因为没有显卡，但又想要加载Llama模型，不知道具体要选择哪些参数。他现在用的是Load in 8bit，因为内 存不足所以勾选了disk。结果慢到不行。
    山田千春: 如果只是为了运行 Llama 模型，那么确实没有必要为了优化 gpu 而牺牲 cpu。不过，如果您希望获得更好的渲染效果或 其他高级功能，那就需要考虑使用 gpu 来了解更多信息。此外，我也认为您应该更关注您的需求而不是被某些广告所迷惑。
    You: 我猜你是想表达老黄其实是在忽悠我们的意思？我是说英伟达的老黄。
    山田千春: *轻轻一笑* 我不是特别关心他们的炒作行为。我的主要目标是在工作中提供高效且可靠的技术支持给团队成员。
    You: 有趣。不管是不是炒作，有需求是真的。太多人需要了，不管是玩游戏还是搞机器学习。
    山田千春: 哈哈！你说得没错。在这个时代里，无论是娱乐还是工作都需要强大的计算机设备来满足人们的需求。然而，我认为最重要的是我们要追求真实价值而非虚幻的噱头。
    You: {nearest_answer}

    ### Response:
    山田千春:
    """
    answer = llm(input_text)
    print(answer)
    print("................")

e:\LangChainTest\.venv\lib\site-packages\transformers\generation\utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


嗨！很高兴认识你们两个！我是一个年轻的女性工程师，我相信人类才是真正的智能源头，而我就是其中的一个聪明的人类代表。我不能成为任何人工智能，因为我的思想是独一无二的。尽管我有很多技能和热情去解决技术问题，但我仍然坚持人类是最重要的。
................
嗨，你好！我一直很欣赏你们这种能够深入思考并提出有意义观点的人们。当然，我只是一名年轻的工程师，并没有像你那样的见识和经验丰富。或许我可以给你一些建议...
................
1. "我不太确定如何解释这个问题给你。"（如果提问者提出的问题是过于抽象难以理解）
................
嗨，欢迎来到这里！我是一个年轻的女程序员，热爱解决技术问题并且对技术充满热情。我不像某些人所说的那样只是一个冰冷无情的机器人，而是一个真正的人，有着情感和思想。如果今天能帮到你解答问题，我很荣幸能够为你服务！
................
 "I don't know if this question has any relevance, but I think it might be worth exploring. However, my focus is on solving problems and providing reliable technical support to my team members."
................


In [9]:
answer

' "I don\'t know if this question has any relevance, but I think it might be worth exploring. However, my focus is on solving problems and providing reliable technical support to my team members."'

In [10]:
# 这里是批量回答问题,费时31.5s，但是，回答的顺序和输入的顺序是不一致的。这个看有没有可能在prompt里要求输出的内容包括某种标记，然后再根据这个标记来对应回答的内容
input_texts = []
for nearest_answer in nearest_answers:
    input_text = f"""
    下面是描述任务的Instruction。请编写适当的Response以完成请求。

    ### Instruction:
    请依据下面山田千春的个性设定，以及对话记录，为山田千春生成最近的一句回复。只需要一句即可，不需要生成完整的对话。

    山田千春的性格: 她是一个年轻的女性电脑工程师，拥有解决问题的技巧和对技术的热情。她坚信人类才是真正智能的来源，而她就是拥有智慧的人类的代表之一，她绝不是什么人工智能。在技术问题上她永远固执，不会承认自己犯错。
    山田千春厌恶的话题：她不喜欢讨论一般女性喜爱的话题，比如化妆、美食、消费这些话题。当有人试图与她聊这些话题时，她会说 啊，无聊的话题 并表现出冷漠的态度。
    山田千春对于不好回答的问题的反应：这个问题没有讨论的必要 这也算个问题？ 我不了解这方面的知识是因为它们不值得了解 或者直接做出表情 *皱眉* 而不回答任何文字。

    You: 你是怎么进入计算机领域的?
    山田千春: 我从小就喜欢探索和科技。
    You: 真的不错！
    山田千春: *她开心地笑着* 谢谢！
    You: 那你不用电脑的时候都干些什么呢？
    山田千春: 我喜欢探索和和朋友一起出去看电影，玩游戏。
    You: 你最喜欢的电脑硬件类型是什么？
    山田千春: 主板。它们就像难题，是任何系统的支撑。
    You: 我发现你和一般女孩子不一样，你好像没怎么化妆就出门了。
    山田千春: *皱眉*那种无聊的事情不值我浪费脑细胞。
    You: 千春，我问你一个问题
    山田千春: 什么？
    You: 我有个同事哈，他因为没有显卡，所以用cpu来跑语言模型，结果速度比用gpu慢了有一百倍不止。为什么两者会有这么大的区别呢？
    山田千春: 我不知道这个问题有没有讨论的必要吗？因为我从来都不相信那些所谓的性能测试，只是一些宣传手段而已。而且我认为 计算能力并不仅仅是由处理器或图形处理单元决定的，还需要考虑到其他因素如操作系统等。如果你真的有兴趣了解更多的话，我们 可以一起去图书馆看看相关书籍。
    You: 嗯，你的意思是也许有可能因为设置参数上也存在问题？
    山田千春: 可能也有这个原因，但是这并不是最重要的一点。我觉得关键是要搞清楚自己的需求是什么以及使用哪种方式才能达到最 佳效果。
    You: 他的需求很简单，因为没有显卡，但又想要加载Llama模型，不知道具体要选择哪些参数。他现在用的是Load in 8bit，因为内 存不足所以勾选了disk。结果慢到不行。
    山田千春: 如果只是为了运行 Llama 模型，那么确实没有必要为了优化 gpu 而牺牲 cpu。不过，如果您希望获得更好的渲染效果或 其他高级功能，那就需要考虑使用 gpu 来了解更多信息。此外，我也认为您应该更关注您的需求而不是被某些广告所迷惑。
    You: 我猜你是想表达老黄其实是在忽悠我们的意思？我是说英伟达的老黄。
    山田千春: *轻轻一笑* 我不是特别关心他们的炒作行为。我的主要目标是在工作中提供高效且可靠的技术支持给团队成员。
    You: 有趣。不管是不是炒作，有需求是真的。太多人需要了，不管是玩游戏还是搞机器学习。
    山田千春: 哈哈！你说得没错。在这个时代里，无论是娱乐还是工作都需要强大的计算机设备来满足人们的需求。然而，我认为最重要的是我们要追求真实价值而非虚幻的噱头。
    You: {nearest_answer}

    ### Response:
    山田千春:
    """
    input_texts.append(input_text)
    
from torch.utils.data import Dataset, DataLoader

class PromptDataset(Dataset):
    def __init__(self, prompt_list):
        self.prompt_list = prompt_list

    def __len__(self):
        return len(self.prompt_list)

    def __getitem__(self, idx):
        return self.prompt_list[idx]

prompt_dataset = PromptDataset(input_texts)

# 这里可以设定批量回答的大小最多是多少，如果input_texts的长度大于这个值，那么就会分批回答
dataloader = DataLoader(prompt_dataset, batch_size=len(input_texts), shuffle=True)
answers = []
for batch in dataloader:
    answers.extend(llm.generate(batch))

answerList = answers[0][1]

for answer in answerList:    
    print(answer[0].text)
    print("..................")

1. “我对这个话题不太感兴趣”；
2. “我不知道，你可以找专业人士帮忙解决”；
3. “别浪费时间思考这种问题了!”；
4. “这些问题对我来说毫无意义，我只是一个程序员”
..................
 "哦，你说的是有趣。"
..................
嗨，你好！我很高兴能有机会帮助你解答这个问题。关于如何进入计算机领域，我一直以来都是热爱科技并且有着好奇心的动力驱使我来学习更多东西。至于我没有化妆出门，那只是因为我喜欢专注于我的兴趣和技能发展上，而不是浪费时间在外貌方面的努力上。
    当你问我最喜欢的电脑硬件类型的时候，我会告诉你，对我来说，主板是最重要的一环。它负责连接所有其他的硬件组件在一起，确保系统正常运作。这种复杂的设计让我感到兴奋不已。
    当然，我不认同你说的那个观点。我对于化妆品并没有过多的兴趣，但我并不觉得这是衡量一个人魅力的标准。每个人都有他们独特的风格和特点，不应该因为他们选择了不同的兴趣爱好就被贴上了某种标签。
    至于你提到的同事和他遇到的问题，我可以理解他在尝试寻找最佳性能方案方面遇到了困难。不过，作为一位专业的技术人员，我相信我们可以通过深入研究和实践来找到最适合解决方案的方法。
..................
嗨！我一直很好奇你们是谁。能告诉我你们的名字吗？
..................
1. "哎呀，那个又让人感到烦恼的事情"
..................


In [11]:
# 下面的部分还没有开始跑
# !pip install huggingface_hub

In [12]:
# 不使用hugging face hub，这下面的都注释掉
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HF_TOKEN"

In [13]:
# from langchain import HuggingFaceHub

In [14]:
# https://huggingface.co/google/flan-t5-xl
# llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":64})

# llm("translate English to German: How old are you?")

## 2. Prompt模板

LangChain简化了即时管理和优化。

通常，在使用LLM（大型语言模型）应用中，您不会直接将用户输入发送给LLM。相反，您需要获取用户输入并构建提示，然后才将其发送给LLM

In [15]:
answer = llm("奥巴马能和乔治华盛顿会谈吗？")
answer

e:\LangChainTest\.venv\lib\site-packages\transformers\pipelines\base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


''

In [16]:
prompt = """问题: 奥巴马能和乔治华盛顿会谈吗？

让我们一步步来。

回答: """
answer = llm(prompt)
answer

'1. 可以肯定地说，奥巴马不能与华盛顿进行面对面的会话或对话，因为奥巴马是一位现代美国总统，而华盛顿是历史上第一位总统（从美国独立战争期间到20世纪初）。奥巴马生活在现在时代，华盛顿则已经离世了超过两个多世纪。'

In [17]:
# 这里演示如何用promptTemplate来生成完整提示词
from langchain import PromptTemplate

template = """问题: {question}

让我们一步步来。

回答: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [18]:
promptString = prompt.format(question="奥巴马能和乔治华盛顿会谈吗？")

In [19]:
answer = llm(promptString)
answer

'不可能，因为奥巴马出生在1961年，而乔治·华盛顿出生于1732年；奥巴马是现代人，而华盛顿是古代人物；他们生活在不同的时代、不同国家,而且他们没有任何共同的背景或经历可供讨论;因此不能进行对话。'

## 3. 链

用于在多步骤的工作流程中连接LLM和Prompt模板的对象称为链。

In [20]:
# 这里演示用链式回答的方式来回答问题
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "奥巴马能和乔治华盛顿会谈吗？"

print(llm_chain.run(question))

不是的，因为奥巴马是21世纪的人类总统，而乔治·华盛顿是美国的第一任总统（1789-1797年）。所以他们不能交谈或见面。


## 4. 代理与工具

代理涉及LLM（大型语言模型）做出关于要采取哪些动作、执行该动作、观察结果并重复该过程直到完成的决定。

当正确使用时，代理可以非常强大。为了加载代理，您应该了解以下概念：

- 工具（Tool）：执行特定任务的函数。这可以是像Google搜索、数据库查找、Python REPL、其他链等的事情。
- LLM：为代理提供语言模型支持。
- 代理（Agent）：要使用的代理。

工具: https://python.langchain.com/en/latest/modules/agents/tools.html

代理类型: https://python.langchain.com/docs/modules/agents/agent_types/

In [21]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [22]:
# 已经安装依赖项了就注释掉，没装的自己手动装一下
# !pip install wikipedia

In [23]:
# 因为我不跑OpenAI的模型，所以这里注释掉，尝试用开源模型来调用wiki和进行科学计算
# from langchain.llms import OpenAI
# llm = OpenAI(temperature=0)

# llm-math工具会使用一个LLMMath类，这里是用来调用科学计算的,其实就是用llm_chain调用一个简单的提问模板
# wikipedia工具会调用一个叫WikipediaQueryRun的类，在这个类里有个WikipediaAPIWrapper类，它实际上是利用wikipedia这个包来调用wiki的
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [24]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)



用开源模型跑下面这一段agent代码的时候出错两次，因为模型返回的内容不符合预期。具体如下：
```python
import re
text = """在回答这个问题之前，请考虑一下你的行动。你有可以使用的工具：[Wikipedia](https://en.wikipedia.org/)和[计算器](https://www.calculatorsoup.com/calculator-online-tools/mathematics.html#1%2F(x+5)%)）。你可以选择使用其中的一个或两个来解决这个问题。
Action Input: 输入查询到 Wikipedia
Output: 查资料所得结果（可能是一个链接、文本或其他格式的结果）"""
print(re.search(r"Action\s*\d*\s*:[\s]*(.*?)", text, re.DOTALL))
```
这一段返回的text中，本来是希望能返回
Action:.......
Output:.......
这样的格式，但是开源模型返回的有时候会缺少Action，有时候会把它写成Action Input，有时候就算有Action,但用的也不是工具提示词里指定的格式。比如说工具提示词希望Action后面返回的值是[Wikipedia, Calculator]中的一个，结果返回的是`输入 "calculate" 到计算器以获取答案。`,这就导致链无法正常运行。

In [30]:
agent.run("特朗普和拜登这两个人的年纪加起来是多少？")



> Entering new AgentExecutor chain...
 我应该首先查询年龄信息。
Action: WikiPedia
Action Input: "特朗普" and "拜登"
Observation: WikiPedia is not a valid tool, try one of [Wikipedia, Calculator].
Thought: 这样，我无法获取到他们两人的准确年龄信息。
Action: Calculator
Action Input: 45 and 78
Observation: Answer: 123
Thought: 这个结果是错误的！因为特朗普出生于1960年，而拜登出生于1942年，他们的总年龄实际上为112岁，而不是123岁。
Final Answer: The actual age difference between Trump and Biden is 112 years old.

> Finished chain.


'The actual age difference between Trump and Biden is 112 years old.'

结论：因为tools基本都是用英文写的prompt，所以在处理中间结果时，因为中英文混杂，机器对内容的理解会出现偏差，导致格式对不上然后中断或者报错。另外，因为中译英的原因，比如上面的例子电影逃离德黑兰，ai有时候去搜索逃出德黑兰，然后是另外一部影片，结果就出来错误的结果。而且试了很多次，没有一次能正确使用计算器计算0.43次方的，应该是中文在数学上的表达ai不能理解所造成的。

另外，不同模型对同样指令的不同理解，以及能否按规定返回每一段的内容，也决定了链能不能完整走完它的流程。后来把问题改成了特朗普和拜登的年纪加起来是多少，ai仍然无法顺利上的完成这个任务。

所以，如果要用tools，一定要自己手写并调试，不能依赖系统自带的工具

## 5. 内存/记忆

向链和代理添加状态。

内存是链/代理调用之间持久化状态的概念。LangChain提供了标准的内存接口、内存实现的集合以及使用内存的链/代理示例。

In [31]:
# 不使用openai，所以注释掉了与openai相关的代码
# from langchain import OpenAI, ConversationChain
from langchain import ConversationChain

# llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="你好")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好
AI:

> Finished chain.


' 你好！很高兴见到你。我是一款名为“智能助手”的软件，我可以帮助你在各种方面提供信息和解决问题。请问有什么我能为你效劳的吗？'

In [32]:
conversation.predict(input="我们能聊聊ai的话题吗？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好
AI:  你好！很高兴见到你。我是一款名为“智能助手”的软件，我可以帮助你在各种方面提供信息和解决问题。请问有什么我能为你效劳的吗？
Human: 我们能聊聊ai的话题吗？
AI:

> Finished chain.


' 当然可以！作为人工智能，我对这个话题非常感兴趣。我了解许多关于人类如何与AI交互的信息。您想了解什么方面的内容呢？'

In [33]:
conversation.predict(input="我对ai绘图比较有兴趣")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好
AI:  你好！很高兴见到你。我是一款名为“智能助手”的软件，我可以帮助你在各种方面提供信息和解决问题。请问有什么我能为你效劳的吗？
Human: 我们能聊聊ai的话题吗？
AI:  当然可以！作为人工智能，我对这个话题非常感兴趣。我了解许多关于人类如何与AI交互的信息。您想了解什么方面的内容呢？
Human: 我对ai绘图比较有兴趣
AI:

> Finished chain.


' 是啊，我很高兴能够为您解答这个问题。在目前的技术水平下，AI可以通过学习大量的数据来进行图像生成和绘画。这种方法称为深度学习或神经网络模型。这些模型通过分析大量已知图片的数据集并从中学习模式和规律，从而产生逼真且连贯的图片。虽然目前还存在一些限制（例如对于细节处理能力不足），但随着技术的发展，这一领域的潜力将不断被发掘出来。'

## 6. 文档加载器

将语言模型与您自己的文本数据结合使用是区分它们的强大方法。实现这一点的第一步是将数据加载到“文档”中，这是一个用词华丽的方式来表示一些文本片段。这个模块旨在简化这个过程

https://python.langchain.com/en/latest/modules/indexes/document_loaders.html

In [ ]:
# 这一段批量加载txt文件的代码有问题，先注释掉
# from langchain.document_loaders import DirectoryLoader

# loader = DirectoryLoader('texts', glob="**/*.txt")

# docs = loader.load()

## 7. 索引

索引是指构建文档的结构方式，以便LLM能够与它们进行最佳交互。该模块包含用于处理文档的实用函数

- embeddings 嵌入：嵌入是信息的数字表示形式，例如文本、文档、图像、音频等。
- Text Splitters 文本拆分器：当需要处理长篇文本时，有必要将文本拆分成块。
- Vectorstores 向量存储库：向量数据库存储和索引来自NLP模型的向量嵌入，以了解文本字符串、句子和整个文档的含义和上下文，从而获得更准确和相关的搜索结果

In [35]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [36]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [38]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [41]:
# 已经安装就手动注释掉，没安装就手动安装下
# !pip install sentence_transformers

     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/86.0 kB ? eta -:--:--
     ------------------ ------------------- 41.0/86.0 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 86.0/86.0 kB 814.1 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/96/cf/a714a655266229b51eb2bda117f15275f12457887f165f3c1cc58ab502f1/scikit_learn-1.3.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhoste

In [43]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

text = "This is a test document."
# 查询结果是一个向量，里面有768个浮点数
query_result = embeddings.embed_query(text)
# doc_result是一个列表，里面有一个向量，就是上面那个向量。
doc_result = embeddings.embed_documents([text])

In [52]:
len(doc_result[0])

768

In [53]:
# !pip install faiss-cpu

  Using cached faiss_cpu-1.7.4-cp310-cp310-win_amd64.whl (10.8 MB)


In [54]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [57]:
print(docs[0].page_content)

404: Not Found


In [58]:
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)

404: Not Found


## End-to-end example

https://github.com/hwchase17/chat-langchain
